In [71]:
#!pip install SQLAlchemy

In [72]:
#!pip install psycopg2
#!pip install psycopg2-binary

In [73]:
from sqlalchemy import create_engine
#import psycopg2


In [97]:
HOST = 'localhost'
PORT = '5433'
USERNAME = 'ahmed'
PASSWORD = '' 
DB = 'northwind'

In [98]:
conn_string = f'postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DB}'

conn_string_mac = f'postgresql+psycopg2://{HOST}:{PORT}/{DB}'

In [99]:
conn_string_mac

'postgresql+psycopg2://localhost:5433/northwind'

In [100]:
engine = create_engine(conn_string_mac)

In [103]:
query = """
CREATE TABLE neworders2 (
col_1 VARCHAR(50),
col_2 INTEGER,
col_3 REAL
);
"""


In [104]:
engine.execute(query)

In [105]:
import pandas as pd

In [106]:
df = pd.read_csv('../../data/orders.csv')

In [107]:
df.head()

,orderID,customerID,employeeID,orderDate,requiredDate,shippedDate,shipVia,freight,shipName,shipAddress,shipCity,shipRegion,shipPostalCode,shipCountry
0,10248,VINET,5,1996-07-04 00:00:00.000,1996-08-01 00:00:00.000,1996-07-16 00:00:00.000,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,NaN,51100,France
1,10249,TOMSP,6,1996-07-05 00:00:00.000,1996-08-16 00:00:00.000,1996-07-10 00:00:00.000,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,NaN,44087,Germany
2,10250,HANAR,4,1996-07-08 00:00:00.000,1996-08-05 00:00:00.000,1996-07-12 00:00:00.000,2,65.83,Hanari Carnes,Rua do Paço 67,Rio de Janeiro,RJ,05454-876,Brazil
3,10251,VICTE,3,1996-07-08 00:00:00.000,1996-08-05 00:00:00.000,1996-07-15 00:00:00.000,1,41.34,Victuailles en stock,2 rue du Commerce,Lyon,NaN,69004,France
4,10252,SUPRD,4,1996-07-09 00:00:00.000,1996-08-06 00:00:00.000,1996-07-11 00:00:00.000,2,51.30,Suprêmes délices,Boulevard Tirou 255,Charleroi,NaN,B-6000,Belgium


In [108]:
df.loc[df["shipCity"]=="Rio de Janeiro","shipCity"] = "Rio" 

# .loc[row, column]
#df.loc[1,2] = "...Value..."

In [109]:
df.head()

,orderID,customerID,employeeID,orderDate,requiredDate,shippedDate,shipVia,freight,shipName,shipAddress,shipCity,shipRegion,shipPostalCode,shipCountry
0,10248,VINET,5,1996-07-04 00:00:00.000,1996-08-01 00:00:00.000,1996-07-16 00:00:00.000,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,NaN,51100,France
1,10249,TOMSP,6,1996-07-05 00:00:00.000,1996-08-16 00:00:00.000,1996-07-10 00:00:00.000,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,NaN,44087,Germany
2,10250,HANAR,4,1996-07-08 00:00:00.000,1996-08-05 00:00:00.000,1996-07-12 00:00:00.000,2,65.83,Hanari Carnes,Rua do Paço 67,Rio,RJ,05454-876,Brazil
3,10251,VICTE,3,1996-07-08 00:00:00.000,1996-08-05 00:00:00.000,1996-07-15 00:00:00.000,1,41.34,Victuailles en stock,2 rue du Commerce,Lyon,NaN,69004,France
4,10252,SUPRD,4,1996-07-09 00:00:00.000,1996-08-06 00:00:00.000,1996-07-11 00:00:00.000,2,51.30,Suprêmes délices,Boulevard Tirou 255,Charleroi,NaN,B-6000,Belgium


In [110]:
df.to_sql('orders4', engine)

First let's see the **SCHEMA** information, schema is where the metadata of the databse is stored.


Select table names 

In [111]:
query_tablenames = """SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES
WHERE table_schema='public';"""
list(engine.execute(query_tablenames))

# Try it with SELECT * too.

[('categories',),
 ('employee_territories',),
 ('employees',),
 ('order_details',),
 ('products',),
 ('regions',),
 ('shippers',),
 ('suppliers',),
 ('territories',),
 ('customers',),
 ('orders',),
 ('employee_success',),
 ('neworders2',),
 ('orders4',)]

select data types of columns

In [112]:
query_column_type = """SELECT COLUMN_NAME,DATA_TYPE  
FROM INFORMATION_SCHEMA.COLUMNS 
WHERE TABLE_NAME='orders4';
"""


In [90]:
list(engine.execute(query_column_type))

[('index', 'bigint'),
 ('orderID', 'bigint'),
 ('customerID', 'text'),
 ('employeeID', 'bigint'),
 ('orderDate', 'text'),
 ('requiredDate', 'text'),
 ('shippedDate', 'text'),
 ('shipVia', 'bigint'),
 ('freight', 'double precision'),
 ('shipName', 'text'),
 ('shipAddress', 'text'),
 ('shipCity', 'text'),
 ('shipRegion', 'text'),
 ('shipPostalCode', 'text'),
 ('shipCountry', 'text')]

In [91]:
query2 = """
ALTER TABLE orders4 ALTER COLUMN "shipName" TYPE VARCHAR(100);
"""

In [92]:
engine.execute(query2)

In [93]:
query3="DROP table orders4"

In [94]:
engine.execute(query3)

Let's check that the table doesn't exist anymore.

In [95]:
query4="SELECT * FROM orders4"
#engine.execute(query4)

Read suppliers table from postgres to pandas dataframe 

In [96]:
query5="SELECT * FROM suppliers"
suppliers = pd.read_sql(query5, engine)
suppliers.head()

,supplierid,companyname,contactname,contacttitle,address,city,region,postalcode,country,phone,fax,homepage
0,1,Exotic Liquids,Charlotte Cooper,Purchasing Manager,49 Gilbert St.,London,NULL,EC1 4SD,UK,(171) 555-2222,NULL,NULL
1,2,New Orleans Cajun Delights,Shelley Burke,Order Administrator,P.O. Box 78934,New Orleans,LA,70117,USA,(100) 555-4822,NULL,#CAJUN.HTM#
2,3,Grandma Kelly's Homestead,Regina Murphy,Sales Representative,707 Oxford Rd.,Ann Arbor,MI,48104,USA,(313) 555-5735,(313) 555-3349,NULL
3,4,Tokyo Traders,Yoshi Nagase,Marketing Manager,9-8 Sekimai Musashino-shi,Tokyo,NULL,100,Japan,(03) 3555-5011,NULL,NULL
4,5,Cooperativa de Quesos 'Las Cabras',Antonio del Valle Saavedra,Export Administrator,Calle del Rosal 4,Oviedo,Asturias,33007,Spain,(98) 598 76 54,NULL,NULL


### Exercises

1. Load the first 3 suppliers from the database to pandas and print the result
2. Load data from a CSV file to the database (pick your choice of data)
3. postgres -> pandas -> postgres
    - Load a table from the database to pandas
    - Add a new column that is based from an old column
    - Save the modified table to a new table name in postgres

References: 

This is a good source when you have a bit more time to read on SQLAlchemy: https://docs.sqlalchemy.org/en/13/core/engines.html

Schema information for POSTGRESQL: https://www.postgresqltutorial.com/postgresql-schema/
https://www.postgresql.org/docs/9.1/ddl-schemas.html